# Creating Household Profiles based on Regulatory Settings

In this notebook, we create optimized household load profiles, based on given regulatory settings. The results are saved in the output folder, where they are later picked up for the grid analysis and the initial analysis of the aggregated profiles.

In [1]:
# IMPORTS
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB, quicksum
import os

In [ ]:
# TARIFF SETTINGS
feed_in_tariff_fixed = 0.1187
grid_charge = 0.0722 # https://www.bundesnetzagentur.de/SharedDocs/Mediathek/Monitoringberichte/Monitoringbericht_VerbraucherKennzahlen2019.pdf
peak_power_charge = 67.94 #https://www.avacon-netz.de/content/dam/revu-global/avacon-netz/documents/netzentgelte-strom
segmented_charges = [grid_charge / 2, grid_charge, grid_charge * 2] # [EUR/kWh segment 1, EUR/kWh segment 2, ...]
segmented_limits = [2, 2, None]   # [kWh segment 1, kWh segment 2, ...]; None for the last segment

In [ ]:
# DEVICE SETTINGS THAT ARE THE SAME FOR ALL HOUSEHOLDS
bess_efficiency = 0.95
guarantee_cycles = 365
factor_min_bess = 0.05  # factor to determine the minimum energy level of the battery
max_blocking_events = 3  # maximum number of blocking events per 24h-window for the heat pump

In [2]:
# DATA
# Household configuration
household_config = pd.read_pickle("./input/preprocessed/2019 Hamelin Household Configuration.pkl")

# 500 EV, HH and HP profiles
df_ev = pd.read_pickle("./input/preprocessed/2019 Hamelin 500 EV.pkl")
df_hh = pd.read_pickle("./input/preprocessed/2019 Hamelin 500 HH.pkl")
df_hp = pd.read_pickle("./input/2019 Hamelin 500 HP.pkl")

# PV generation for 1kW nominal capacity in Hamelin; Data created with https://renewables.ninja/
df_pv = pd.read_csv("./input/ninja_pv_52.1040_9.3562_uncorrected.csv",skiprows=3)

# Price data; we use day-ahead German spot market prices
df_price = pd.read_excel("./input/Gro_handelspreise_201901010000_201912312359_Stunde.xlsx", skiprows=9) # Note: price data is not converted to UTC, in contrast to EV/HH/HP/PV (CET)
df_p = pd.DataFrame(index=df_ev.index)
# MWh prices are transformed to kWh prices
df_p["Deutschland/Luxemburg [€/kWh]"] = df_price["Deutschland/Luxemburg [€/MWh]"].apply(lambda x: x/1000).values
min_price = df_p["Deutschland/Luxemburg [€/kWh]"].min()
df_p["Deutschland/Luxemburg [€/kWh]"] = df_p["Deutschland/Luxemburg [€/kWh]"].apply(lambda x: x+abs(min_price)) # avoid negative values for optimization
df_p.head()

# Number of time steps and days
n_ts = len(df_p)
n_days = int(n_ts/24)

/Users/leosemmelmann/opt/anaconda3/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
# HELPER FUNCTIONS
def get_results_in_df(m, variableNames, n_timesteps):
    """
    Transform optimization results into a DataFrame.

    :param m: Gurobi model
    :param variableNames: list of variable names to consider
    :param n_timesteps: number of time steps
    :return: DataFrame with optimization results; columns - variables, rows - time steps
    """
    # Initialize
    results_df = pd.DataFrame(columns=variableNames, index=[t for t in range(n_timesteps)])
    # Iterate over all variables and time steps
    for n in variableNames:
        for t in range(n_timesteps):
            results_df.loc[t][n] = m.getVarByName(n + f"[{t}]").x
    return results_df

def get_rotating_grid_charges(n_ts, volumetric_charge, idx_initial):
    """
    Get grid charges for the rotating grid charge type.

    :param n_ts:  number of time steps
    :param volumetric_charge: constant volumetric grid charge
    :param idx_initial:  initial index of the household
    :return: Array with grid charge for each time step
    """
    modulo_value = idx_initial%2
    gc = np.full(n_ts, volumetric_charge/2)
    for i in range(modulo_value, n_ts, 2):
        gc[i] = volumetric_charge * 2
    return gc

In [ ]:
# OPTIMIZATION
# Settings
debug = False  # if True, the results are not saved in the output folder
household_n = 500  # household number must be less than or equal to 500
opt_mip_gap = 0.01  # threshold MIP gap
opt_time_limit = 3000  # time limit for optimization in seconds
operation_type = "dynamic"  # "dynamic" or "constant"
ev_charging_strategy = "early"  # "early" or "spread"; only relevant for operation_type = "constant"
    # operation_type    |   ev_charging_strategy    |   explanation
    # constant          |   early                   |   immediate charging when EV is plugged in
    # constant          |   spread                  |   EV charging is spread over the entire plug-in time

# Iterate over various scenarios; the lists determine which scenarios are covered
for pricing_type in ["constant", "dynamic"]:  # ["constant", "dynamic"]
    for grid_charge_type in ["volumetric", "peak", "segmented", "rotating"]:  # ["volumetric", "peak", "segmented", "rotating"]:
        for feed_in_type in ["fit", "dynamic"]:  # ["fit", "dynamic"]
            for grid_charging_allowed in [False]:  # [True, False]
                result_path = f"./output/00_pricing_{pricing_type}_operation_{operation_type}_fi_{feed_in_type}_ne_{str(grid_charge_type)}_gridch_{str(grid_charging_allowed)}.pkl"
                print(f"Starting optimization for pricing_{pricing_type}_operation_{operation_type}_fi_{feed_in_type}_ne_{str(grid_charge_type)}_gridch_{str(grid_charging_allowed)}")
                
                if debug is False:
                    if os.path.exists(result_path):
                        df_results = pd.read_pickle(result_path)
                    else:
                        df_results = pd.DataFrame()
                        df_results.index = df_p.index
                else:
                    df_results = pd.DataFrame()
                    df_results.index = df_p.index

                # Iterate over households
                for idx_initial, household in household_config.iloc[:household_n].iterrows():
                    if str(idx_initial) not in df_results.columns: # check if already optimized that household
                        # Load data for the household
                        hp_load = df_hp[household["heat_pump_profile"]]
                        hh_load = df_hh[household["household_profile"]]
                        ev_load = df_ev[household["household_profile"]].copy().round(5)  # round to avoid numerical issues
                        pv_size = household["pv_power"]
                        bess_size = household["bess_capacity"]
                        max_bess_power = household["bess_power"]

                        # Transform EV load to kW and calculate maximum ev charging
                        ev_load["kWh"] = ev_load["Wh"].apply(lambda x:x/1000).values
                        max_ev_charging = (ev_load["kWh"] / ev_load["share_of_hour"]).max()
                        # Transform HH and HP load to kW and calculate maximum HP load
                        real_hh_load = hh_load.apply(lambda x:x/1000).values
                        real_hp_load = hp_load.apply(lambda x:x/1000).values
                        max_hp_load = real_hp_load.max()

                        # Other device settings
                        pv_size = bess_size
                        pv_load = df_pv["electricity"].apply(lambda x: x*pv_size).values
                        min_bess_energy = factor_min_bess*bess_size

                        # Initialize environment and model
                        env = gp.Env(empty=True)
                        env.setParam("OutputFlag", 1)
                        env.start()
                        model = gp.Model("model", env=env)
                        model.setParam("MIPGap", opt_mip_gap)
                        model.setParam("TimeLimit", opt_time_limit)

                        # DETERMINING PRICES BASED ON TYPE
                        if pricing_type == "dynamic":
                            prices = df_p["Deutschland/Luxemburg [€/kWh]"].apply(lambda x: x).values
                        elif pricing_type == "constant":
                            average_price = df_p["Deutschland/Luxemburg [€/kWh]"].mean()
                            prices = np.full(n_ts, average_price)
                        else:
                            raise ValueError("Pricing type not defined.")
                            
                        # DETERMINING GRID CHARGE FOR EACH TIME STEP BASED ON TYPE
                        if grid_charge_type == "volumetric":
                            grid_charges  = np.full(n_ts, grid_charge)
                        elif grid_charge_type =="peak":
                            grid_charges = np.full(n_ts, 0)
                        elif grid_charge_type == "segmented":
                            grid_charges = np.full(n_ts, 0)  # set the default grid charge to zero; the cost for each segment will be added later on
                        elif grid_charge_type == "rotating":
                            grid_charges = get_rotating_grid_charges(n_ts, grid_charge, idx_initial)
                        else:
                            raise ValueError("Grid charge type not properly defined.")

                        # DETERMINING FEED-IN REMUNERATION
                        if feed_in_type == "fit":
                            feed_in_tariff = np.full(n_ts, feed_in_tariff_fixed)
                        elif feed_in_type == "dynamic":
                            feed_in_tariff = df_p["Deutschland/Luxemburg [€/kWh]"].values
                        elif feed_in_type == "zero":
                            feed_in_tariff = np.full(n_ts, 0)
                        else:
                            raise ValueError("Feed-in remuneration not defined.")

                        # VARIABLES; names start with "v_"
                        opt_ev_charging = model.addVars([t for t in range(n_ts)], lb=0, vtype=GRB.CONTINUOUS, name="v_ev_charging")
                        opt_hp_load = model.addVars([t for t in range(n_ts)], lb=0, vtype=GRB.CONTINUOUS, name="v_hp_load")
                        opt_bess_charging = model.addVars([t for t in range(n_ts)], lb=0, ub=max_bess_power, vtype=GRB.CONTINUOUS, name="v_bess_charging")
                        opt_bess_discharging = model.addVars([t for t in range(n_ts)], lb=0, ub=max_bess_power, vtype=GRB.CONTINUOUS, name="v_bess_discharging")
                        opt_net_energy = model.addVars([t for t in range(n_ts)], lb=0, vtype=GRB.CONTINUOUS, name="v_net_energy")
                        soe_bess = model.addVars([t for t in range(n_ts)], lb=min_bess_energy, ub=bess_size, vtype=GRB.CONTINUOUS, name="v_soe_bess")

                        opt_feedin_pv = model.addVars([t for t in range(n_ts)], lb=0, vtype=GRB.CONTINUOUS, name="v_feedin_pv")
                        opt_internaluse_pv = model.addVars([t for t in range(n_ts)], lb=0, vtype=GRB.CONTINUOUS, name="v_internaluse_pv")

                        energy_costs = model.addVars([t for t in range(n_ts)], vtype=GRB.CONTINUOUS,name="v_energy_costs")
                        feedin_profits = model.addVars([t for t in range(n_ts)], vtype=GRB.CONTINUOUS,name="v_feedin_profits")

                        block_hp = model.addVars([t for t in range(n_ts)], vtype=gp.GRB.BINARY, name="v_block_hp")
                        block_hp_hour = model.addVars([t for t in range(n_ts)], vtype=gp.GRB.BINARY, name="v_block_hp_hour")

                        # Auxiliary variables for particular regulatory cases
                        if grid_charge_type == "peak":
                            max_net_energy = model.addVar(name="v_max_net_energy")
                        elif grid_charge_type == "segmented":
                            energy_per_segment = model.addVars([t for t in range(n_ts)], [s for s in range(len(segmented_charges))], lb=0, vtype=GRB.CONTINUOUS, name="v_energy_per_segment")  # upper bounds are set in the constraints

                        # CONSTRAINTS; names start with "c_"; fixed bounds are set in the variable definitions
                        # ELECTRIC VEHICLE
                        # Ensure that charging requirements are met
                        if operation_type == "dynamic":
                            for ts_start in ev_load[ev_load["start"] > 0].index:
                                hours_until_end = int(ev_load.loc[ts_start, "hours_until_end"])
                                # ts_start is the timestamp when the charging session starts; convert to integer index
                                idx_start = ev_load.index.get_loc(ts_start)
                                idx_end = idx_start + hours_until_end
                                # Ensure that the energy demand of each charging session is met
                                model.addConstr(quicksum(opt_ev_charging[t] for t in range(idx_start, idx_end + 1)) ==
                                                quicksum(ev_load.iloc[t]["kWh"] for t in range(idx_start, idx_end + 1)), f"c_ev_energy[{idx_start}]")
                                                # idx_end must be included --> + 1

                            # Ensure that the maximum empirical EV charging is never exceeded
                            # If the EV is only plugged in for a fraction of the hour (i.e., share_of_hour < 1),
                            # it can also only charge for that fraction
                            model.addConstrs((opt_ev_charging[t] <= max_ev_charging * ev_load.iloc[t]["share_of_hour"] for t in range(n_ts)), name="c_ev_max_charging")
                        elif operation_type == "constant" and ev_charging_strategy == "early":
                            # Run helper optimization to define charging power in each time step
                            model_h = gp.Model("ev_early_charging", env=env)
                            opt_ev_charging_h = model_h.addVars([t for t in range(n_ts)], vtype=GRB.CONTINUOUS, name="opt_ev_charging_h")
                            coeff_early_charging = [0] * len(ev_load)  # used in the objective function
                            for ts_start in ev_load[ev_load["start"] > 0].index:
                                hours_until_end = int(ev_load.loc[ts_start, "hours_until_end"])
                                # ts_start is the timestamp when the charging session starts ; convert to integer index
                                idx_start = ev_load.index.get_loc(ts_start)
                                idx_end = idx_start + hours_until_end
                                # ensure that the energy demand of each charging session is met
                                model_h.addConstr(quicksum(opt_ev_charging_h[t] for t in range(idx_start, idx_end + 1)) ==
                                                quicksum(ev_load.iloc[t]["kWh"] for t in range(idx_start, idx_end + 1)))
                                coeff_early_charging[idx_start:idx_end + 1] = range(hours_until_end + 1)

                            # Ensure maximum empirical EV charging is never exceeded
                            model_h.addConstrs(opt_ev_charging_h[t] <= max_ev_charging * ev_load.iloc[t]["share_of_hour"] for t in range(n_ts))

                            # Objective function
                            model_h.setObjective(quicksum(opt_ev_charging_h[t] * coeff_early_charging[t] for t in range(n_ts)), GRB.MINIMIZE)
                            model_h.optimize()

                            # Assign results to the main model
                            model.addConstrs((opt_ev_charging[t] == opt_ev_charging_h[t].x for t in range(n_ts)), name="c_ev_charging_constant")
                        else:
                            # EV load is spread over the entire plug-in time; this is how the input data is formatted
                            model.addConstrs((opt_ev_charging[t] == ev_load.iloc[t]["kWh"] for t in range(n_ts)), name="c_ev_charging_constant")

                        # HEAT PUMP
                        if operation_type == "dynamic":
                            # Limits on heat pump load in each time step
                            model.addConstrs((opt_hp_load[t] >= (1 - block_hp_hour[t]) * real_hp_load[t] for t in range(n_ts)), name="c_hp_load_lb")
                            model.addConstrs((opt_hp_load[t] <= (1 - block_hp_hour[t]) * max_hp_load for t in range(n_ts)), name="c_hp_load_ub")
                            # Counter of blocking events; block_hp = 1 if there is a switch from blocked to unblocked
                            model.addConstrs((block_hp[t] >= block_hp_hour[t-1] - block_hp_hour[t] for t in range(1, n_ts)), name="c_hp_end_blocking_event")
                            # Limit the number of blocking events per 24h-window
                            model.addConstrs((quicksum(block_hp[t] for t in range(k, k+24)) <= max_blocking_events for k in range(n_ts-23)), name="c_hp_max_blocking_events_per_24h")
                            # After a blocking event, the HP must be unblocked for at least 2 hours
                            model.addConstrs((block_hp_hour[t] + block_hp_hour[t+1] <= 2 * (1-block_hp[t]) for t in range(n_ts-1)), name="hp_min_unblock")
                            # The HP can at most be blocked for 2 consecutive hours
                            model.addConstrs((block_hp_hour[t] + block_hp_hour[t+1] + block_hp_hour[t+2] <= 2 for t in range(n_ts-2)), name="hp_max_block")
                            # Ensure that the sum of the heat pump load in the given 6-hour intervals remains the same
                            for idx in range(n_days * 4):
                                idx_start = idx * 6
                                idx_end = (idx + 1) * 6
                                if idx_end > n_ts:
                                    idx_end = n_ts  # Ensure that we do not exceed the bounds
                                model.addConstr(quicksum(opt_hp_load[t] for t in range(idx_start, idx_end)) == quicksum(real_hp_load[t] for t in range(idx_start, idx_end)), name=f"c_hp_load_6hr_block[{idx}]")
                        elif operation_type == "constant":
                            model.addConstrs((opt_hp_load[t] == real_hp_load[t] for t in range(n_ts)), name="c_hp_load_constant")

                        # BATTERY STORAGE
                        if grid_charging_allowed is not True:
                            model.addConstrs((opt_bess_charging[t] <= opt_internaluse_pv[t] for t in range(n_ts)), name="c_bess_charging_ub_pv")
                        model.addConstrs((opt_bess_charging[t] * opt_bess_discharging[t] == 0 for t in range(n_ts)), name="c_bess_mutual_exclusivity")
                        for t in range(n_ts):
                            soe_prev = min_bess_energy if t == 0 else soe_bess[t-1]
                            model.addConstr(soe_bess[t] == soe_prev+opt_bess_charging[t]*bess_efficiency-opt_bess_discharging[t]/bess_efficiency, name=f"c_bess_soe_evolution[{t}]")
                        model.addConstr(quicksum(opt_bess_discharging[t] + opt_bess_charging[t] for t in range(n_ts)) <= bess_size*2*guarantee_cycles, name="c_bess_guarantee_cycles")

                        # NET ENERGY CONSUMPTION OF THE HOUSEHOLD
                        model.addConstrs((opt_net_energy[t] == (real_hh_load[t]
                                                                + opt_ev_charging[t]
                                                                + opt_hp_load[t]
                                                                + opt_bess_charging[t] - opt_bess_discharging[t]
                                                                - opt_internaluse_pv[t]) for t in range(n_ts)), name="c_net_energy")

                        # PV
                        model.addConstrs(((opt_internaluse_pv[t] + opt_feedin_pv[t]) == pv_load[t] for t in range(n_ts)), name="c_pv_balance")
                        model.addConstrs((opt_net_energy[t] * opt_feedin_pv[t] == 0 for t in range(n_ts)), name="c_pv_mutual_exclusivity")

                        # AUXILIARY VARIABLES FOR PARTICULAR REGULATORY CASES
                        if grid_charge_type == "peak":
                            model.addConstrs((max_net_energy >= opt_net_energy[t] for t in range(n_ts)), name=f"c_max_net_energy")
                        elif grid_charge_type == "segmented":
                            model.addConstrs((energy_per_segment[t,s] <= segmented_limits[s] for t in range(n_ts) for s in range(len(segmented_charges) - 1)), name="c_upper_bound_segment")  # no upper bound for the last segment
                            model.addConstrs((quicksum(energy_per_segment[t,s] for s in range(len(segmented_charges))) == opt_net_energy[t] for t in range(n_ts)), name="c_sum_energy_per_segment")

                        # COSTS AND FEED-IN REVENUES
                        model.addConstrs((energy_costs[t] == opt_net_energy[t] * (prices[t]+grid_charges[t]) for t in range(n_ts)), "c_costs")
                        model.addConstrs((feedin_profits[t] == opt_feedin_pv[t] * feed_in_tariff[t] for t in range(n_ts)), "c_revenues")

                        # OBJECTIVE FUNCTION
                        if grid_charge_type == "peak":
                            model.setObjective(quicksum(energy_costs[t]-feedin_profits[t] for t in range(n_ts)) + max_net_energy*peak_power_charge, GRB.MINIMIZE)
                        elif grid_charge_type == "segmented":
                            model.setObjective(quicksum(energy_costs[t]-feedin_profits[t] for t in range(n_ts)) + quicksum(energy_per_segment[t,s]*segmented_charges[s] for t in range(n_ts) for s in range(len(segmented_charges))), GRB.MINIMIZE)
                        else:
                            model.setObjective(quicksum(energy_costs[t]-feedin_profits[t] for t in range(n_ts)), GRB.MINIMIZE)

                        # OPTIMIZE
                        # Write to file to enable analysis in case of infeasibilities
                        model.update()
                        model.write("model.mps")
                        model.optimize()

                        # GET RESULTS
                        vn = ["v_ev_charging", "v_hp_load", "v_bess_charging", "v_bess_discharging", "v_soe_bess", "v_net_energy", "v_feedin_pv", "v_internaluse_pv", "v_energy_costs", "v_feedin_profits", "v_block_hp", "v_block_hp_hour"]
                        temp_results = get_results_in_df(model, vn, n_ts)

                        # SAVE RESULTS
                        df_results[str(idx_initial)] = temp_results["v_net_energy"].values - temp_results["v_feedin_pv"].values
                        if debug is False:
                                df_results.to_pickle(result_path)
                    else:
                        print(str(idx_initial)+" has been already optimized for that case.")